## Stationary calibration

This notebook contains the results of calibrating the 4 phones when configured next to each other with the same settings and placed next to each other in the same bookcase.

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import branca.element as bre

## Load and validate data

The first issue to note is that we actually have two specs here. The first spec is the checked in `evaluation.spec.sample`, which defines calibration for both stationary and moving instances, and some evaluation trips. However, while starting with the calibration, we noticed some inconsistencies between the power curves. So in order to be more consistent, I defined a second, calibration-only spec `examples/calibration.only.json`, which essentially repeats the calibration experiments multiple times.

After that, I returned to the first set of experiments for the moving calibration and the evaluation.

In [ ]:
DATASTORE_URL = "http://localhost:8080"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_hf = eisd.ServerSpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "sfba_hf_calibration_stationary_only")
sd_mf = eisd.ServerSpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "sfba_med_freq_calibration_stationary_only")

In [ ]:
pv_hf = eipv.PhoneView(sd_hf)

In [ ]:
pv_mf = eipv.PhoneView(sd_mf)

In [ ]:
import importlib
importlib.reload(ezpv)

This validation fails because we forgot to multiply the filter_time by 1000 before setting it.
So we set the value to 1 ms when we meant to set it to 1 sec.
This is not a super bad issue since:
- it only affects android
- due to built-in throttling the data is actually returned only at 1sec frequency or even less anyway
- we don't use this to model anything else, we just use it to help choose regimes for further testing, and for checking that the drain is consistent

We had fixed this bug before the medium frequency collection (phew!) so that does validate properly

In [ ]:
# pv_hf.validate()
# pv_mf.validate()

In [ ]:
ev_hf = eiev.EvaluationView()
ev_hf.from_view_multiple_runs(pv_hf, "")

In [ ]:
ev_mf = eiev.EvaluationView()
ev_mf.from_view_multiple_runs(pv_mf, "")

## Visualize battery drain

### Basic visualization

Dump everything into one giant graph

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(16,16))

ezpv.plot_all_power_drain(ios_ax, pv_hf.map()["ios"], "calibration", "stationary")
ezpv.plot_all_power_drain(ios_ax, pv_mf.map()["ios"], "calibration", "stationary")
ios_ax.legend(loc="upper left", mode="expand", bbox_to_anchor=(0, 3.75, 1.25,0.2), ncol=2, prop={"size": 12})
ezpv.plot_all_power_drain(android_ax, pv_hf.map()["android"], "calibration", "stationary")
ezpv.plot_all_power_drain(android_ax, pv_mf.map()["android"], "calibration", "stationary")
android_ax.legend(loc="lower left", mode="expand", bbox_to_anchor=(0,-3,1.25,0.2), ncol=2, prop={"size": 12})

### Grouped by sensing configuration

If the previous graph is too busy, this graph groups the values from similar experiments together so that we can see whether they separate out or not. We can clearly see two clusters of lines for both android and iOS.

- For android, the only really high power mode is high frequency high accuracy. Lowering either the frequency (e.g. `best_high_accuracy_medium_freq_stationary` or the accuracy `100m_balanced_accuracy_medium_freq_stationary`) makes the power significantly different and by about the same value.
- For iOS, the the frequency does not seem to make any difference. The only variation is caused by lowering the accuracy to 100m

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(10,10))

color_map = {}
legend_map = {}

(color_map, legend_map) = ezpv.plot_collapsed_all_power_drain(ios_ax, pv_hf.map()["ios"], "calibration", "stationary", color_map, legend_map)
(color_map, legend_map) = ezpv.plot_collapsed_all_power_drain(ios_ax, pv_mf.map()["ios"], "calibration", "stationary", color_map, legend_map)
(color_map, legend_map) = ezpv.plot_collapsed_all_power_drain(android_ax, pv_hf.map()["android"], "calibration", "stationary", color_map, legend_map)
(color_map, legend_map) = ezpv.plot_collapsed_all_power_drain(android_ax, pv_mf.map()["android"], "calibration", "stationary", color_map, legend_map)
ios_ax.legend(legend_map.values(), legend_map.keys())
android_ax.legend(legend_map.values(), legend_map.keys())

### Separate plots to see individual variations

In case we want to see individual variations in a less overwhelming way, we can plot the individual runs in separate plots.

#### Phone based plots

In [ ]:
(ifig, ax) = plt.subplots(figsize=(12,3), nrows=0, ncols=0)
ezpv.plot_separate_power_drain(ifig, pv_hf.map()["ios"], 4, "calibration", "stationary")
ezpv.plot_separate_power_drain(ifig, pv_mf.map()["ios"], 4, "calibration", "stationary")
(ifig, ax) = plt.subplots(figsize=(12,3), nrows=0, ncols=0)
ezpv.plot_separate_power_drain(ifig, pv_hf.map()["android"], 4, "calibration", "stationary")
ezpv.plot_separate_power_drain(ifig, pv_mf.map()["android"], 4, "calibration", "stationary")

In [ ]:
importlib.reload(ezev)

#### Experiment based plots

In [ ]:
(ifig, ax) = plt.subplots(figsize=(12,12), nrows=0, ncols=0)
ezev.plot_separate_power_drain_multiple_runs(ifig, 1, ev_hf.map("calibration")["ios"], "")

In [ ]:
(ifig, ax) = plt.subplots(figsize=(12,12), nrows=0, ncols=0)
ezev.plot_separate_power_drain_multiple_runs(ifig, 1, ev_mf.map("calibration")["android"], "")

## Checking counts

We now check the number of data points collected during calibration and their distribution in an effort to validate the duty cycling. Observations from this are:

##### on android: more points = more power drain

As we would expect, the number of points across the various phones and the various runs is almost identical. In the cases where it is significantly different (e.g. `high-accuracy-stationary-0` on `ucb-sdb-android-1` and `high-accuracy-stationary-3` on `ucb-sdb-android-3`), we have see significant differences in the power drain as well. However, we do not understand why these two runs behave differently from the other runs.

##### on iOS: almost no points

Since iOS has a distance filter, and not a time filter, and this calibration was stationary, almost no points are generated for high accuracy sensing. However, with the 100m sensing, we do get a significant number of points (an order of magnitude more), although nowhere near the number of entries on android.

##### on android: medium accuracy = almost no points

On android, medium accuracy sensing generates two orders of magnitude fewer points than high accuracy. So the additional power drain on android probably reflects not just the sensing cost but also the processing cost. This also indicates that the medium accuracy sensing, which relies on WiFi and cellular signal strengths, is likely to be suspended when the phone is in doze mode.

In [ ]:
hf_count_df = ezpv.get_count_df(pv_hf); hf_count_df

In [ ]:
mf_count_df = ezpv.get_count_df(pv_mf); mf_count_df

In [ ]:
(ifig, ax) = plt.subplots(nrows=1, ncols=3, figsize=(16,6))
hf_count_df.filter(like="best_high_accuracy").filter(like="android", axis=0).plot(ax=ax[0],kind="bar")
hf_count_df.filter(like="ios", axis=0).plot(ax=ax[1],kind="bar")
hf_count_df.filter(like="10m_balanced").filter(like="android", axis=0).plot(ax=ax[2],kind="bar")

In [ ]:
(ifig, ax) = plt.subplots(nrows=1, ncols=3, figsize=(16,6))
mf_count_df.filter(like="best_high_accuracy").filter(like="android", axis=0).plot(ax=ax[0],kind="bar")
mf_count_df.filter(like="ios", axis=0).plot(ax=ax[1],kind="bar")
mf_count_df.filter(like="100m_balanced").filter(like="android", axis=0).plot(ax=ax[2],kind="bar")

## Checking densities

In [ ]:
android_density_df = ezpv.filter_density_df(ezpv.get_location_density_df(pv_hf.map()["android"], "calibration"))
android_ax = android_density_df.plot(kind='density', subplots=False, figsize=(10,10), sharex=True, sharey=True)
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
android_density_df.rename(columns={"ucb-sdb-android-1_best_high_accuracy_high_frequency_stationary_0": "phone 1 run 0",
                                   "ucb-sdb-android-1_best_high_accuracy_high_frequency_stationary_1": "phone 1 run 1",
                                   "ucb-sdb-android-2_best_high_accuracy_high_frequency_stationary_0": "phone 2 run 0"}, inplace=True)

In [ ]:
(ifig, axes) = plt.subplots(figsize=(10,5), nrows=1, ncols=2)
pv_hf.map()["android"]["ucb-sdb-android-1"]["calibration_ranges"][0]["battery_df"].plot(ax=axes[0], x="hr", y="battery_level_pct", ylim=(50,100), label="phone 1 run 0")
pv_hf.map()["android"]["ucb-sdb-android-1"]["calibration_ranges"][1]["battery_df"].plot(ax=axes[0], x="hr", y="battery_level_pct", ylim=(50,100), label="phone 1 run 1")
pv_hf.map()["android"]["ucb-sdb-android-2"]["calibration_ranges"][0]["battery_df"].plot(ax=axes[0], x="hr", y="battery_level_pct", ylim=(50,100), label="phone 2 run 0")
android_density_df[["phone 1 run 0", "phone 1 run 1", "phone 2 run 0"]].plot(kind='density', ax=axes[1])
axes[0].set_ylabel("Battery level")
axes[0].set_xlabel("Hour")
# axes[0].set_xlim(0,20)
axes[1].set_ylabel("Location point density")
axes[1].set_xlabel("Hour")
axes[1].set_xlim(-5,20)

In [ ]:
android_density_df.columns

In [ ]:
android_density_df = ezpv.filter_density_df(ezpv.get_location_density_df(pv_mf.map()["android"], "calibration"))
android_ax = android_density_df.plot(kind='density', subplots=False, figsize=(10,10), sharex=True, sharey=True)
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
ios_density_df = ezpv.filter_density_df(ezpv.get_location_density_df(pv_hf.map()["ios"], "calibration"))
ios_ax = ios_density_df.plot(kind='density', subplots=False, figsize=(10,10), sharex=True, sharey=True)
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
ios_density_df = ezpv.filter_density_df(ezpv.get_location_density_df(pv_mf.map()["ios"], "calibration"))
ios_ax = ios_density_df.plot(kind='density', subplots=False, figsize=(10,10), sharex=True, sharey=True)
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))